In [ ]:
# Import necessary libraries for making HTTP requests, random selection, and data manipulation
import requests
import random
import pandas as pd

In [ ]:
# Prompt user for Upstox API credentials and redirect URI for authentication
api_key = input('\tapi_key\t')
secret_key = input('\tsecret_key\t')
redirect_uri = input('\tredirect_uri\t')
# Construct the login URL with the client ID (api_key) and redirect URI
login_url = f'https://api.upstox.com/v2/login/authorization/dialog?client_id={api_key}&redirect_uri={redirect_uri}'
print(login_url)# Print the URL so the user can visit it to authenticate and retrieve the authorization code

# Prompt user to enter the authorization code received from the login URL
code = input('\tcode\t')

# Set up the URL, headers, and data for the POST request to obtain the access token
api__auth__url = 'https://api.upstox.com/v2/login/authorization/token'
api__auth__hdrs= {
    'accept': 'application/json',
    'Content-Type': 'application/x-www-form-urlencoded'
}
api__auth__data = {
    'code': code,
    'client_id': api_key,
    'client_secret': secret_key,
    'redirect_uri': redirect_uri,
    'grant_type': 'authorization_code'
}

# Send the request to get the access token
auth__req_res = requests.request('POST',api__auth__url, headers=api__auth__hdrs, data=api__auth__data)

# Try to retrieve and save the access token; if it exists, reuse it
try:
    access_token = auth__req_res.json()['access_token']
    with open('access_token.txt', 'w') as f:
        f.write(access_token)
except:
    print(f"'<access_token>' already generated for the code '{code}'")

# Read the access token from file for use in future API requests
access_token = open('access_token.txt', 'r').read()

https://api.upstox.com/v2/login/authorization/dialog?client_id=7a0e0d79-7477-4ea4-8e01-a1a8f2cd674a&redirect_uri=https://www.upstox.com


In [12]:
# Define the URL for retrieving the complete instrument data in JSON format
api__mtqe_ints_exage_cmplte__url = "https://assets.upstox.com/market-quote/instruments/exchange/complete.json.gz"

# Load the JSON data into a DataFrame containing all available instruments
mtqe_ints_exage_cmplte__df = pd.read_json(api__mtqe_ints_exage_cmplte__url)

# Filter the DataFrame to include only options of type 'PE' (put) or 'CE' (call)
mtqe_ints_exage_cmplte__df = mtqe_ints_exage_cmplte__df[(mtqe_ints_exage_cmplte__df['instrument_type'] == 'PE') | (mtqe_ints_exage_cmplte__df['instrument_type'] == 'CE')]

# Convert the expiry column to a date format for easier selection
mtqe_ints_exage_cmplte__df['expiry'] = pd.to_datetime(mtqe_ints_exage_cmplte__df['expiry'], unit='ms').dt.date.astype(str)

# Define the URL and headers to retrieve long-term portfolio holdings using the access token
api__prtflo_lgtmhs__url = "https://api.upstox.com/v2/portfolio/long-term-holdings"
api__prtflo_lgtmhs__hdrs = {
        'Authorization' : f'Bearer {access_token}',
        'Accept': 'application/json'
    }

# Make the API request to retrieve portfolio data and convert the response to JSON
prtflo_lgtmhs__req_res = requests.request("GET", api__prtflo_lgtmhs__url, headers=api__prtflo_lgtmhs__hdrs).json()

# Convert the portfolio JSON data to a DataFrame with relevant columns
prtflo_lgtmhs__df = pd.DataFrame(prtflo_lgtmhs__req_res['data'])[['instrument_token','product','quantity']]

# Merge the option chain data with portfolio data based on matching keys, and drop unnecessary columns
mtqe_ints_exage_cmplte_prtflo_lgtmhs__df = pd.merge(mtqe_ints_exage_cmplte__df, prtflo_lgtmhs__df, left_on='underlying_key', right_on='instrument_token').drop(columns=['underlying_key','instrument_token'])

# Insert an 'instrument_key' column at the start of the DataFrame for clarity
mtqe_ints_exage_cmplte_prtflo_lgtmhs__df.columns.to_list().insert(0, 'instrument_key')


In [13]:
# Function to retrieve option chain data for a given instrument, expiry date, and option side
def func__get_option_chain_data(args__instrument_name: str, args__expiry_date: str, args__side: str) -> pd.DataFrame:
    # Filter the data for matching instrument name, expiry date, and side to get the instrument key
    instrument_key__df = mtqe_ints_exage_cmplte__df[(mtqe_ints_exage_cmplte__df['name'] == args__instrument_name) & (mtqe_ints_exage_cmplte__df['expiry'] == args__expiry_date) & (mtqe_ints_exage_cmplte__df['instrument_type'] == args__side)]['underlying_key'].to_list()
    
    # Select an instrument key, either through input or by random choice
    instrument_key = input('\tinstrument_key\t') or random.choice(instrument_key__df)
    
    # Define the URL, headers, and parameters to retrieve the option chain data for the selected instrument key
    api__opt_chn__url = "https://api.upstox.com/v2/option/chain"
    api__opt_chn__hdrs = {
        'Authorization' : f'Bearer {access_token}',
        'Accept': 'application/json'
    }
    api__opt_chn__prms = {
        'instrument_key': instrument_key,
        'expiry_date': mtqe_ints_exage_cmplte__df[mtqe_ints_exage_cmplte__df['underlying_key'] == instrument_key]['expiry']
    }
    
    # Make the API request and retrieve the option chain data in JSON format
    opt_chn__req_res = requests.request("GET", api__opt_chn__url, headers=api__opt_chn__hdrs, params=api__opt_chn__prms).json()
    
    # Construct a DataFrame with option chain details: instrument name, average strike price, side, and highest bid or ask price
    ittn_prdct_qunty__df = pd.DataFrame({
        'instrument_name': [args__instrument_name],
        'strike_price': [mtqe_ints_exage_cmplte__df[mtqe_ints_exage_cmplte__df['underlying_key'] == instrument_key]['strike_price'].mean()],
        'side': [args__side],
        'bid/ask': [max(itr__data["put_options"]["market_data"]["bid_price"] for itr__data in opt_chn__req_res['data']) if args__side == 'PE' else max(itr__data["call_options"]["market_data"]["ask_price"] for itr__data in opt_chn__req_res['data'])]
    })
    return ittn_prdct_qunty__df

# Function to calculate margin and premium based on the retrieved option chain data
def func__calculate_margin_and_premium(args__data: pd.DataFrame) -> pd.DataFrame:
    # Define the URL, headers, and JSON payload to request margin calculation from the API
    api__crgs_mrgn__url = "https://api.upstox.com/v2/charges/margin"
    api__crgs_mrgn__hdrs = {
        "accept": "application/json",
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    api__crgs_mrgn__json = {
        "instruments": [
            {
                "instrument_key": prtflo_lgtmhs__df['instrument_token'].values[0],
                "product": prtflo_lgtmhs__df['product'].values[0],
                "quantity": int(prtflo_lgtmhs__df['quantity'].values[0]),
                "transaction_type": "SELL",   
            }
        ]
    }
    
    # Send the POST request to calculate the margin requirement and retrieve the response
    crgs_mrgn__req_res = requests.request("POST", api__crgs_mrgn__url, headers=api__crgs_mrgn__hdrs, json=api__crgs_mrgn__json)
    
    # Calculate margin required and premium earned based on retrieved data
    mnrd_pmed__lst = [crgs_mrgn__req_res.json()['data']['required_margin'], args__data['bid/ask'].values[0] * mtqe_ints_exage_cmplte__df[mtqe_ints_exage_cmplte__df['underlying_key'] == prtflo_lgtmhs__df['instrument_token'].values[0]]['lot_size'].values[0]]
    ittn_prdct_qunty_mnrd_pmed__df = args__data.assign(**{'margin_required':mnrd_pmed__lst[0],'premium_earned':mnrd_pmed__lst[1]})
    return ittn_prdct_qunty_mnrd_pmed__df


In [14]:
# Initialize empty DataFrames to store results
gocd__df__op = pd.DataFrame()
cmad__df__op = pd.DataFrame()

# Loop to allow multiple queries until an empty input is given
while(input(">>> || <<<") != ''):
    # Retrieve unique instrument names for user selection
    instrument_names__lst = mtqe_ints_exage_cmplte_prtflo_lgtmhs__df['name'].unique().tolist()
    print(instrument_names__lst)
    
    # Prompt user for instrument name and default to random if not provided
    instrument_name__io = input("\tinstrument_name\t") or random.choice(instrument_names__lst)
    
    # Retrieve unique expiry dates for selected instrument
    expiry_date__lst = mtqe_ints_exage_cmplte_prtflo_lgtmhs__df[mtqe_ints_exage_cmplte_prtflo_lgtmhs__df['name'] == instrument_name__io]['expiry'].unique().tolist()
    print(expiry_date__lst)
    expiry_date__io = input("\texpiry_date\t") or random.choice(expiry_date__lst)
    
    # Retrieve available option sides (PE/CE) for selected expiry date
    side__lst = mtqe_ints_exage_cmplte_prtflo_lgtmhs__df[mtqe_ints_exage_cmplte_prtflo_lgtmhs__df['expiry'] == expiry_date__io]['instrument_type'].unique()
    print(side__lst)
    side__io = input("\tside\t") or random.choice(side__lst)
    
    # Display selected instrument details
    print(instrument_name__io, expiry_date__io, side__io)
    
    # Fetch option chain data and display the result
    gocd__df__ip = func__get_option_chain_data(instrument_name__io, expiry_date__io, side__io)
    gocd__df__op = pd.concat([gocd__df__op, gocd__df__ip], ignore_index=True).iloc[::-1]
    display(gocd__df__op)

    # Calculate margin and premium, then display the result
    cmad__df__ip = func__calculate_margin_and_premium(gocd__df__ip)
    cmad__df__op = pd.concat([cmad__df__op, cmad__df__ip], ignore_index=True).iloc[::-1]
    display(cmad__df__op)
else:
    pass  # Exit the loop when no input is provided


['VODAFONE IDEA LIMITED']
['2024-12-26', '2025-01-30', '2024-11-28']
['CE' 'PE']
VODAFONE IDEA LIMITED 2025-01-30 CE


,instrument_name,strike_price,side,bid/ask
0,VODAFONE IDEA LIMITED,9.846154,CE,23.9


,instrument_name,strike_price,side,bid/ask,margin_required,premium_earned
0,VODAFONE IDEA LIMITED,9.846154,CE,23.9,0.0,956000.0
